In [6]:
import nltk
import random
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split

In [7]:
with open('BIG_BOT_CONFIG.json','r') as f:
    BOT_CONFIG=json.load(f)

In [8]:
def cleaner(text): 
    cleaned_text = ''
    for ch in text.lower():
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz ':
            cleaned_text = cleaned_text + ch
    return cleaned_text

def match(text, example): 
    return nltk.edit_distance(text, example) / len(example) < 0.4 if len(example) > 0 else False

def get_intent(text): 
    for intent in BOT_CONFIG['intents']:
        if 'examples' in BOT_CONFIG['intents'][intent]:
             for example in BOT_CONFIG['intents'][intent]['examples']:
                if match(cleaner(text), cleaner(example)):
                       return intent

In [9]:
X = []
y = []

for intent in BOT_CONFIG['intents']:
     if 'examples' in BOT_CONFIG['intents'][intent]:
        X += BOT_CONFIG['intents'][intent]['examples']
        y += [intent for i in range(len(BOT_CONFIG['intents'][intent]['examples']))]

In [10]:
vectorizer = CountVectorizer(preprocessor=cleaner, ngram_range=(1,3), stop_words=['а', 'и'])

In [11]:
vectorizer.fit(X)
X_vect = vectorizer.transform(X)

In [12]:
X_train_vect, X_test_vect, y_train, y_test = train_test_split(X_vect, y, test_size=0.3)

In [13]:
sgd = SGDClassifier() 
sgd.fit(X_vect, y)

SGDClassifier()

In [14]:
sgd.score(X_vect, y)

0.782725321888412

In [15]:
def get_intent_by_model(text): # Функция определяющая интент текста с помощью ML-модели
     return sgd.predict(vectorizer.transform([text]))[0]

In [16]:
def bot(text): # функция бота
    intent = get_intent(text)  # 1. попытаться понять намерение сравнением по Левинштейну
    
    if intent is None:
        intent = get_intent_by_model(text) # 2. попытаться понять намерение с помощью ML-модели

    return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [17]:
import logging

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help_command(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""
    text = update.message.text
    print(text)
    reply = bot(text)
    update.message.reply_text(reply)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1772897111:AAGdJcxzGiQiM8erkEf-0K44TK7I72NqG48")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on noncommand i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()

2021-03-25 22:30:05,401 - apscheduler.scheduler - INFO - Scheduler started


Привет
Как дела?
Что?
Ты чего?
Как дела?
Как дела?
Нормально
Хорошо
Отлично
Норм
Я поел
Спасибо брат
?
Где?
ну
не надо
да
Привет
Как жизнь?
Бывает, что делаешь?
Справедливо
Есть такое
Кто?
Красава
Правда?
Что мне делать?
Погода ?
